In [1]:
import pandas as pd

In [2]:
purchase_csv = 'https://opendata-ajuntament.barcelona.cat/data/dataset/1144c2b2-43a5-48f6-b447-5f11a3a34b87/resource/e42cf2cf-a76e-4a32-9357-cf90e0ea8ead/download/locveevolucio.csv'
rent_csv='https://opendata-ajuntament.barcelona.cat/data/dataset/fedae1fc-bd48-4676-afdc-979423786512/resource/97356d26-30b9-436a-8dbb-d0d05f0a87fd/download/loclloevolucio.csv'
business_list='https://opendata-ajuntament.barcelona.cat/data/dataset/62fb990e-4cc3-457a-aea1-497604e15659/resource/c897c912-0f3c-4463-bdf2-a67ee97786ac/download/2019_censcomercialbcn_detall.csv'
geojson_data= 'https://opendata-ajuntament.barcelona.cat/resources/bcn/EstadisticaUnitatsAdministratives/BarcelonaCiutat_Barris.json'

In [3]:
purchase = pd.read_csv(purchase_csv)
rent = pd.read_csv(rent_csv)
business = pd.read_csv(business_list)
geojson = pd.read_json(geojson_data)

C:\Users\Sbort\AppData\Local\Temp\ipykernel_21796\2741577662.py:3: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  business = pd.read_csv(business_list)


In [4]:
purchase['codi_barri'] = purchase.BARRIS.apply(lambda x: x.split('.')[0])

In [5]:
rent['codi_barri'] = rent.BARRIS.apply(lambda x: x.split('.')[0])

In [6]:
rent['codi_barri'] = rent['codi_barri'].astype(int)
purchase['codi_barri'] = purchase['codi_barri'].astype(int)

In [7]:
def price(row):
    if row['DOSZEROONZE'] != '--':
        return row['DOSZEROONZE']
    elif row['DOSZERODEU'] != '--':
        return row['DOSZERODEU']
    elif row['DOSZEROZERONOU'] !=  '--':
        return row['DOSZEROZERONOU']
    elif row['DOSZEROZEROVUIT'] !=  '--':
        return row['DOSZEROZEROVUIT']
    else:
        return None                  

In [8]:
rent['p_lloguer'] = rent.apply(price, axis=1)
purchase['p_venta'] = purchase.apply(price, axis=1)

In [10]:
barrios = rent[['codi_barri','p_lloguer']].merge(purchase[['codi_barri','p_venta']], on='codi_barri', how='left')

In [11]:
barrios2 = business[['Nom_Barri', 'Codi_Barri','Codi_Districte']].drop_duplicates()

In [12]:
barrios2 = barrios2.rename(columns=lambda x: x.lower())

In [13]:
barrios = barrios.merge(barrios2, on='codi_barri', how='left')

In [14]:
geojson = geojson[['codi_barri', 'geometria_wgs84']]

In [15]:
barrios = barrios.merge(geojson, on="codi_barri", how="left")

In [16]:
barrios['p_venta'] = barrios['p_venta'].str.replace('.', '', regex=False)

In [17]:
barrios['p_venta'] = barrios['p_venta'].astype(float)
barrios['p_lloguer'] = barrios['p_lloguer'].astype(float)

In [18]:
barrios['p_venta_rel'] = ((barrios['p_venta'] - barrios['p_venta'].mean())/barrios['p_venta'].mean())*100

In [19]:
barrios['p_lloguer_rel'] = ((barrios['p_lloguer'] - barrios['p_lloguer'].mean())/barrios['p_lloguer'].mean())*100

In [22]:
barrios.to_csv('../data/df_barris.csv')